In [1]:
# Make sure we have the packages we need
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression # package for fitting the regression model later

np.random.seed(123456789)

In [2]:
# Assume underlying stock price follows Geometric Brownian Motion(GBM)
# set up parameters
reps = 10000 # number of simulations
T = 5 # American option maturity date (in annum)
dt = 1/244 # time difference factor (244 trading days per annum)
time = np.arange(0,T,dt) # time points
N = len(time) # number of time intervals
r = 0.06 # riskfree interest rate
sigma = 0.3 # volatility or "randomness"
S_0 = 1 # initial stock price
K = 1.05 # strike price of the American option

In [3]:
# contruct the Geometric Brownian Motion(GBM) by random walk construction
# simulate the underlying price until maturity date T across each discretized time point dt
def underlying_price(reps, N, S_0, r, sigma, dt):    
    '''
    Simulate the underlying stock price paths by Geometric Brownian motion with mean "r" and volatility "sigma".

    Args:
        reps: number of simulated underlying price paths generated by Geometric Brownian Motion
        N: number of time intervals until maturity
        S_0: initial stock price
        r: (annualized) riskfree interest rate (mean of the GBM)
        sigma: (annualized) volatility (standard deviation) of the GBM
        dt: time difference factor
        
    Return:
        S: the matrix of the underlying stock price paths given in each discretized time point (in numpy.array)
    
    '''
    
    S = np.empty([reps,N+1]) # 'reps' sample paths of GBM(r,sigma) movement

    for i in range(reps):
            
        one_path = S_0*np.cumprod(np.exp((r-(sigma**2)/2)*dt + sigma*np.sqrt(dt)*np.random.normal(0,1,N))) # one GBM increment
        
        one_path = np.insert(one_path,0,S_0) # add the initial stock price at inception
        
        S[i,] = one_path

    return S

In [4]:
S = underlying_price(reps, N, S_0, r, sigma, dt)
pd.DataFrame(S)

,0,1,2,3,4,5,6,7,8,9,...,1211,1212,1213,1214,1215,1216,1217,1218,1219,1220
0,1.0,1.043480,1.087085,1.126294,1.128147,1.146982,1.128999,1.154445,1.185430,1.207170,...,0.844239,0.833275,0.786389,0.785456,0.782724,0.804001,0.810296,0.807451,0.795648,0.791993
1,1.0,0.972777,0.963598,0.996611,1.019570,1.041585,1.036654,1.030832,1.049591,1.062676,...,1.041844,1.005448,1.005176,1.017505,1.018500,1.007638,0.967796,0.965637,0.951850,0.945940
2,1.0,0.987726,1.009465,1.028775,0.976480,0.986053,0.999665,0.999226,0.970007,0.966883,...,1.356702,1.366226,1.404218,1.453309,1.458472,1.474322,1.425783,1.477987,1.456470,1.422436
3,1.0,0.989219,0.993299,0.979758,0.992833,1.008770,0.999503,0.992891,0.999234,1.006080,...,1.354639,1.358040,1.375188,1.378368,1.370705,1.369490,1.407653,1.360632,1.354283,1.352489
4,1.0,0.985275,0.979323,0.947698,0.961868,0.953990,0.953147,0.932162,0.924751,0.947748,...,3.229121,3.280261,3.234315,3.303365,3.395835,3.306122,3.301395,3.315793,3.245965,3.085420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,1.022286,1.019080,0.998451,1.011667,1.015404,1.020670,1.001242,1.042496,1.044983,...,1.850090,1.842281,1.885230,1.892971,1.901772,1.903651,1.872040,1.950151,1.887215,1.915304
9996,1.0,0.999051,0.980043,0.954554,0.949287,0.976328,0.993592,0.960721,0.955227,0.956718,...,0.586831,0.579817,0.601291,0.610498,0.589297,0.587931,0.597461,0.593344,0.589856,0.582637
9997,1.0,1.014473,1.031365,1.046305,1.032187,1.001285,0.993216,1.005098,1.001623,1.002407,...,1.449198,1.426020,1.445647,1.449944,1.466026,1.500747,1.500835,1.486759,1.459341,1.459313
9998,1.0,0.994518,1.014909,1.024244,1.030560,1.055605,1.046545,1.064123,1.069395,1.092579,...,0.572603,0.588886,0.603489,0.618810,0.607558,0.617438,0.601635,0.598177,0.598094,0.590708


In [5]:
# approximate each path's discounted conditional expectation value by least square regression
# inspired by Longstaff-Schwartz(2001) Least Sqaure Monte Carlo approach
# a simple LSM approach for pricing put option
def least_square_monte_carlo(reps, N, K, r, dt, S):   
    '''
    Inspired by the Longstaff-Schwartz(2001) Least Sqaure Monte Carlo approach, this function builds a simple and 
    generalized pricing framework for the American (put) option by working in backwardation. 
    
    Args:
        reps: number of simulated underlying stock paths generated by Geometric Brownian Motion
        N: number of time intervals until maturity
        K: strike price for American option
        r: (annualized) riskfree interest rate
        dt: time difference factor
        S: simulated paths generated by Geometric Brownian Motion (in numpy.array)
    
    Return:
        value: the value (cash flow) matrix of the American (put) option given in each discretized time point (in numpy.array)
    
    '''
    
    value = np.empty([reps,N+1]) # final optimal American option cash flow matrix
    
    for i in range(N,-1,-1):
        
        S_current = S[:,i] # current stock price across all paths
        
        if i == N: # determine if exercise the option at maturity
            
            value[:,i] = np.abs((K - S_current)*(K > S_current)) # store the put option payoff if greater than 0
            
        elif i == 0: # determine the discounted conditional expectation value at inception
            
            value[:,i] = np.exp(-r*dt)*value[:,i+1]
            
        else: # do backwardation until the inception
                         
            in_the_money_path_loc = ((K - S_current)*(K > S_current)).nonzero() # determine the in-the-money sample paths location where immediate exercise value > 0
            
            length = len(in_the_money_path_loc[0])
            
            exercise_value = ((K - S_current)*(K > S_current))[in_the_money_path_loc] # immediately exercise value at current node
            
            X = np.empty([length,2]) # the regressor X is the in-the-money path's underlying price at current node
            
            for j in range(1,3):
                
                X[:,j-1] = np.power(S_current[in_the_money_path_loc],j) # our predictor variable X along with the basis function X^2 in LSE regression setup
            
            previous_continuation_value = value[:,i+1][in_the_money_path_loc] # the conditional expectation value in previous node
    
            Y = np.exp(-r*dt)*previous_continuation_value # our target variable Y (discounted value from continuation) in LSE regression setup
            
            try:
            
                least_square_reg_model = LinearRegression().fit(X, Y) # fit the LSE regression to estimate the discounted conditional expectation value
                
            except ValueError:
                
                raise Exception("At the time point (node)", i, "all paths are not optimal to convert into shares")
            
            predicted_conditional_exp = least_square_reg_model.predict(X) # predict the discounted (continued) conditional expectation value
        
            value[:,i] = np.exp(-r*dt)*value[:,i+1]# if do not early exercise the put option, the value of current path will just be the discounted continued option value from the previous node
            
            exercise_immediate_loc = (exercise_value > predicted_conditional_exp) # the optimal early exercise sample path location determined by LSM algorithm 
            
            optimal_loc = in_the_money_path_loc[0][exercise_immediate_loc] # optimal sample paths location at which we should early exercise
            
            optimal_cash_flow = exercise_value[exercise_immediate_loc] # optimal cash flow at current node
            
            value[optimal_loc,(i+1):] = 0 # the early exercise sample paths will have 0 value after the current time point (node) since the option can only be exercised once
            
            value[optimal_loc,i] = optimal_cash_flow # update the current optimal early exercise sample paths' value

    return value

In [6]:
# set up parameters (the American put option example used in paper)
reps = 8 # number of simulations
T = 3 #1 # maturity date
dt = 1 #2**(-10) # time difference factor
time = np.arange(0,T,dt) # time points
N = len(time) # number of time intervals
r = 0.06 # riskfree interest rate
sigma = 0.3 # volatility or "randomness"
S_0 = 1 # initial stock price
K = 1.1 # strike price of the American put option

In [7]:
# test random generated underlying price movement (the American put option example used in paper)
S = np.array([[1,1.09,1.08,1.34],
              [1,1.16,1.26,1.54],
              [1,1.22,1.07,1.03],
              [1,0.93,0.97,0.92],
              [1,1.11,1.56,1.52],
              [1,0.76,0.77,0.9],
              [1,0.92,0.84,1.01],
              [1,0.88,1.22,1.34]])
pd.DataFrame(S)

,0,1,2,3
0,1.0,1.09,1.08,1.34
1,1.0,1.16,1.26,1.54
2,1.0,1.22,1.07,1.03
3,1.0,0.93,0.97,0.92
4,1.0,1.11,1.56,1.52
5,1.0,0.76,0.77,0.90
6,1.0,0.92,0.84,1.01
7,1.0,0.88,1.22,1.34


In [8]:
value = least_square_monte_carlo(reps, N, K, r, dt, S)
pd.DataFrame(value) # cash flow matrix of the American put option across time

,0,1,2,3
0,0.000000,0.000000,0.000000,0.00
1,0.000000,0.000000,0.000000,0.00
2,0.058469,0.062084,0.065924,0.07
3,0.160100,0.170000,0.000000,0.00
4,0.000000,0.000000,0.000000,0.00
5,0.320200,0.340000,0.000000,0.00
6,0.169518,0.180000,0.000000,0.00
7,0.207188,0.220000,0.000000,0.00


In [9]:
np.mean(value[:,0]) # the example American put option price estimated by LSM algorithm

0.11443433004505696

In [10]:
np.max(value[:,0])

0.32019994141864466

In [11]:
# test runtime for the self-defined function
%reload_ext line_profiler
%lprun -f least_square_monte_carlo least_square_monte_carlo(reps, N, K, r, dt, S)

In [ ]:
# # python debugger
# import pdb
# pdb.set_trace()